In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [2]:
import logging


logging.basicConfig(
    level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s'
)

In [3]:
from math_rag.infrastructure.containers import InfrastructureContainer


RESET = False

infrastructure_container = InfrastructureContainer()
infrastructure_container.init_resources()

math_article_seeder = infrastructure_container.math_article_seeder()
math_expression_seeder = infrastructure_container.math_expression_seeder()
math_expression_classification_seeder = (
    infrastructure_container.math_expression_classification_seeder()
)
math_article_seeder.seed(reset=RESET)
await math_expression_seeder.seed(reset=RESET)
await math_expression_classification_seeder.seed(reset=RESET)

math_expression_repository = infrastructure_container.math_expression_repository()
math_expression_classification_repository = (
    infrastructure_container.math_expression_classification_repository()
)
openai_managed_llm = infrastructure_container.openai_managed_llm()

In [ ]:
# TODO
# - description for each class
# - how to determine classes?
# - do names need to have a single token?

In [4]:
math_expressions = await math_expression_repository.find_many()

In [ ]:
from math_rag.application.models.assistants import MathExpressionLabelerAssistantInput


math_expression_labeler_assistant = (
    infrastructure_container.math_expression_labeler_assistant()
)
input = MathExpressionLabelerAssistantInput(latex='a + 2')

output = await math_expression_labeler_assistant.assist(input)